# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [3]:
import findspark
findspark.init()

In [4]:
spark_url = 'local'

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [6]:
sc = spark.sparkContext

In [7]:
spark

In [8]:
sc

<SparkContext master=local appName=Spark Tutorial>

In [9]:
path='/content/netflix-rotten-tomatoes-metacritic-imdb.csv'
df = spark.read.option("delimiter", ",").option("header", True).csv(path)


In [10]:
df.show(3)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|              

In [11]:
df.columns

['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series or Movie',
 'Hidden Gem Score',
 'Country Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View Rating',
 'IMDb Score',
 'Rotten Tomatoes Score',
 'Metacritic Score',
 'Awards Received',
 'Awards Nominated For',
 'Boxoffice',
 'Release Date',
 'Netflix Release Date',
 'Production House',
 'Netflix Link',
 'IMDb Link',
 'Summary',
 'IMDb Votes',
 'Image',
 'Poster',
 'TMDb Trailer',
 'Trailer Site']

## What is the maximum and average of the overall hidden gem score?

In [14]:
from pyspark.sql.functions import avg, min, max, countDistinct,col
df.select(max('hidden gem score'),avg('hidden gem score')).show()

+---------------------+---------------------+
|max(hidden gem score)|avg(hidden gem score)|
+---------------------+---------------------+
|                  9.8|    5.937551386501226|
+---------------------+---------------------+



## How many movies that are available in Korea?

In [15]:
df.filter(col('Languages').like('%Korea%')).select('Languages').count()

735

## Which director has the highest average hidden gem score?

In [16]:
dfdirector=df.groupby('Director').agg({'hidden gem score':'avg'})

In [17]:
dfdirector.select(max('avg(hidden gem score)')).show()

+--------------------------+
|max(avg(hidden gem score))|
+--------------------------+
|                       9.8|
+--------------------------+



In [18]:
dfdirector.filter(dfdirector['avg(hidden gem score)']==9.8).select('Director').show()

+-----------+
|   Director|
+-----------+
|Dorin Marcu|
+-----------+



## How many genres are there in the dataset?

In [19]:
from pyspark.sql.functions import split
dfgenre=df.select(split(col("Genre"),", "))

In [20]:
allgenre=list()
genre=dfgenre.collect()
for i in range(len(genre)):
  genres=genre[i][0]
  if(genres is None):
    continue
  for splitgenre in genres:
    if splitgenre not in allgenre:
      allgenre.append(splitgenre)
print(allgenre)
print(len(allgenre))

['Crime', 'Drama', 'Fantasy', 'Horror', 'Romance', 'Comedy', 'Thriller', 'Animation', 'Short', 'Family', 'Mystery', 'Action', 'Adventure', 'Sci-Fi', 'Music', 'Biography', 'Sport', 'War', 'History', 'Documentary', 'Film-Noir', 'Musical', 'Game-Show', 'Western', 'Reality-TV', 'Talk-Show', 'News', 'Adult']
28
